### Settings and Imports

In [1]:
import numpy as np
import numpy.linalg as lin
import numpy.random as rnd
import scipy as sp
import scipy.optimize as opt
import matplotlib as mpl
import matplotlib.pyplot as plt
import random

from sklearn import svm
from sklearn.model_selection import RepeatedKFold

%matplotlib inline
%precision 4
np.set_printoptions(precision=3)

## Prepare tests

In [2]:
data_train = np.loadtxt("features.train.txt")
x_train, f_train = data_train[:, 1:], data_train[:, 0]
data_test = np.loadtxt("features.test.txt")
x_test, f_test = data_test[:, 1:], data_test[:, 0]

In [3]:
# test arrays for "1 vs 5" tests

def select_vals15(x, f, val1=1, val2=5):
    cond = np.logical_or(f == val1, f == val2)
    x_sel, f_sel = x[cond], f[cond]
    y_sel = np.where(f_sel == val1, [1.], [-1.])
    return x_sel.copy(), y_sel.copy()

x_train15, y_train15 = select_vals15(x_train, f_train, 1, 5)
x_test15, y_test15 = select_vals15(x_test, f_test, 1, 5)

## SKLEARN

In [4]:
# tasks 2,3,4
def val_vs_all(f_val, Q=2, C=0.01):
    y_train = np.where(f_train == f_val, [1.], [-1.])
    svc = svm.SVC(C=C, kernel='poly', degree=Q, gamma=1, coef0=1)
    svc.fit(x_train, y_train)
    nsv = svc.n_support_.sum()
    ein = np.mean(svc.predict(x_train) != y_train)
    print('val={:d} C={:.0e} Q={:d} ein={:.4f} nsv={:<3d}'
          .format(f_val, C, Q, ein, nsv))
    return dict(val=f_val, C=C, Q=Q, nsv=nsv, ein=ein)

In [5]:
task2_res = {val: val_vs_all(val) for val in (0,2,4,6,8)}

val=0 C=1e-02 Q=2 ein=0.1059 nsv=2179
val=2 C=1e-02 Q=2 ein=0.1003 nsv=1970
val=4 C=1e-02 Q=2 ein=0.0894 nsv=1856
val=6 C=1e-02 Q=2 ein=0.0911 nsv=1893
val=8 C=1e-02 Q=2 ein=0.0743 nsv=1776


In [6]:
# task 2  <a>
max((d['ein'],v) for v,d in task2_res.items())

(0.1059, 0)

In [7]:
task3_res = {val: val_vs_all(val) for val in (1,3,5,7,9)}

val=1 C=1e-02 Q=2 ein=0.0144 nsv=386
val=3 C=1e-02 Q=2 ein=0.0902 nsv=1950
val=5 C=1e-02 Q=2 ein=0.0763 nsv=1585
val=7 C=1e-02 Q=2 ein=0.0885 nsv=1704
val=9 C=1e-02 Q=2 ein=0.0883 nsv=1978


In [8]:
# task 3  <a>
min((d['ein'],v) for v,d in task3_res.items())

(0.0144, 1)

In [9]:
# task 4  <c>
task2_res[0]['nsv'] - task3_res[1]['nsv']

1793

In [10]:
# tasks 5,6
def run_sel15(Q, C):
    svc = svm.SVC(C=C, kernel='poly', degree=Q, gamma=1, coef0=1)
    svc.fit(x_train15, y_train15)
    ein = np.mean(svc.predict(x_train15) != y_train15)
    eout = np.mean(svc.predict(x_test15) != y_test15)
    nsv = svc.n_support_.sum()
    print('C={:.0e} Q={:d} ein={:.4f} eout={:.4f} nsv={:<3d}'
          .format(C, Q, ein, eout, nsv))
    return dict(C=C, Q=Q, nsv=nsv, ein=ein, eout=eout)

In [11]:
# task 5  <d>
task5_sel = [run_sel15(Q=2, C=C) for C in (0.001, 0.01, 0.1, 1)]

C=1e-03 Q=2 ein=0.0045 eout=0.0165 nsv=76 
C=1e-02 Q=2 ein=0.0045 eout=0.0189 nsv=34 
C=1e-01 Q=2 ein=0.0045 eout=0.0189 nsv=24 
C=1e+00 Q=2 ein=0.0032 eout=0.0189 nsv=24 


In [12]:
# task 6  <b>
task6_sel = [run_sel15(Q=Q, C=C) for C in (0.0001, 0.001, 0.01, 1) for Q in (2,5)]

C=1e-04 Q=2 ein=0.0090 eout=0.0165 nsv=236
C=1e-04 Q=5 ein=0.0045 eout=0.0189 nsv=26 
C=1e-03 Q=2 ein=0.0045 eout=0.0165 nsv=76 
C=1e-03 Q=5 ein=0.0045 eout=0.0212 nsv=25 
C=1e-02 Q=2 ein=0.0045 eout=0.0189 nsv=34 
C=1e-02 Q=5 ein=0.0038 eout=0.0212 nsv=23 
C=1e+00 Q=2 ein=0.0032 eout=0.0189 nsv=24 
C=1e+00 Q=5 ein=0.0032 eout=0.0212 nsv=21 


In [13]:
def task_7_8(n_runs=10, n_folds=10, random_seed=None):
    x_all, y_all = x_train15, y_train15
    Q = 2
    c_all = [0.0001, 0.001, 0.01, 0.1, 1]
    c_count = {c:0  for c in c_all}
    c_ecv   = {c:[] for c in c_all}

    rkfold = RepeatedKFold(n_splits=n_folds,
                           n_repeats=n_runs*len(c_all),
                           random_state=random_seed)
    split_iter = rkfold.split(list(range(x_all.shape[0])))

    for run_i in range(n_runs):
        c_min, ecv_min = None, np.inf
        for c in c_all:
            ecv_for_splits = []
            for split_i in range(n_folds):
                train_idx, test_idx = next(split_iter)
                x_train, x_test = x_all[train_idx], x_all[test_idx]
                y_train, y_test = y_all[train_idx], y_all[test_idx]
                svc = svm.SVC(C=c, kernel='poly', degree=Q, gamma=1, coef0=1)
                svc.fit(x_train, y_train)
                ecv = np.mean(svc.predict(x_test) != y_test)
                c_ecv[c].append(ecv)
                ecv_for_splits.append(ecv)
            ecv_avg = np.mean(ecv_for_splits)
            if ecv_avg < ecv_min:
                c_min, ecv_min = c, ecv_avg
        c_count[c_min] += 1
    for c in c_all:
        print('C={:.4f} Ecv={:.5f} Nc={:<3d}'
              .format(c, np.mean(c_ecv[c]), c_count[c]))

task_7_8(n_runs=100, random_seed=4913)
# task 7  <c> !!!
# task 8  <c>

C=0.0001 Ecv=0.00983 Nc=0  
C=0.0010 Ecv=0.00482 Nc=28 
C=0.0100 Ecv=0.00470 Nc=35 
C=0.1000 Ecv=0.00481 Nc=24 
C=1.0000 Ecv=0.00489 Nc=13 


In [14]:
def task_9_10():
    for C in (0.01, 1, 100, 1e4, 1e6):
        svc = svm.SVC(C=C, kernel='rbf', gamma=1)
        svc.fit(x_train15, y_train15)
        ein = np.mean(svc.predict(x_train15) != y_train15)
        eout = np.mean(svc.predict(x_test15) != y_test15)
        print('C={:1.0e} ein={:.4f} eout={:.4f}'.format(C, ein, eout))
task_9_10()
# task 9   <e>
# task 10  <c>

C=1e-02 ein=0.0038 eout=0.0236
C=1e+00 ein=0.0045 eout=0.0212
C=1e+02 ein=0.0032 eout=0.0189
C=1e+04 ein=0.0026 eout=0.0236
C=1e+06 ein=0.0006 eout=0.0236


## TODO: CVXPY

In [15]:
import cvxpy

def svm_cvxpy_dual(x, y, verbose=False):
    from cvxpy import Variable, Problem, Minimize
    from cvxpy import sum_entries, quad_form

    atol = 1e-6
    n = len(x)
    yx = y[:,None] * x
    K = yx @ yx.T     # a.T*K*a == quad_form(a,K)
    a = Variable(n)   # alphas
    objective = Minimize(0.5 * quad_form(a, K) - sum_entries(a))
    constraints = [a >= 0, y.T @ a == 0]
    qp = Problem(objective, constraints)
    qp.solve(verbose=verbose)

    aval = np.array(a.value).squeeze(1)
    cond = np.abs(aval) > atol
    w_ = np.sum(x * (aval * y)[:,None], axis=0)
    b_ = (y - x @ w_)[cond]
    w = np.hstack((b_[0], w_))
    idx = list(cond.nonzero()[0])
    sv = x[idx]
    sv_y = sv @ w_ + b_

    return dict(status=qp.status, ok=(qp.status == 'optimal'),
                weights=w, alphas=aval[cond], atol=atol,
                n_sv=len(idx), sv_i=idx, sv=sv, sv_y=sv_y)

## TODO: CVXOPT

In [16]:
import cvxopt

def svm_cvxopt_dual(x, y, verbose=False):
    from cvxopt import matrix, solvers

    atol = 1e-6
    n = x.shape[0]
    y_ = y[:, None]
    yx = y_ * x
    K = yx @ yx.T

    P = matrix(K)
    q = matrix(-np.ones([n,1]))
    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))
    A = matrix(y_.T)
    b = matrix(np.zeros(1))

    solvers.options['show_progress'] = verbose
    qp = solvers.qp(P, q, G, h, A, b)

    assert qp['x'].size == (n, 1)
    aval = np.array(qp['x']).squeeze(1)       # alphas
    cond = np.abs(aval) > atol
    w_ = np.sum(x * (aval * y)[:,None], axis=0)
    b_ = (y - x @ w_)[cond]
    w = np.hstack([b_[0], w_])
    idx = list(cond.nonzero()[0])
    sv = x[idx]
    sv_y = sv @ w_ + b_

    return dict(status=qp['status'], ok=(qp['status'] == 'optimal'),
                weights=w, alphas=aval[cond], atol=atol,
                n_sv=len(idx), sv_i=idx, sv=sv, sv_y=sv_y)